<a href="https://www.kaggle.com/code/nuranynovita/cohort-analysis-customer-retention?scriptVersionId=143746573" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

Cohort analysis is a type of behavioral analytics that can help you understand the health of your business and the loyalty of your customers. To build a cohort, customers are divided into cohorts or groups based on similarities to those groups and specific traits. The term cohort refers to a time-span grouping that divides people according to the week or month they were first acquired.

There are two ways to break the group of users into cohorts based on [Clevertap.com](https://clevertap.com/blog/cohort-analysis/):

* **Acquisition Cohorts**: divide users by when they signed up first for your product. For your app users, you might break down your cohorts by the day, the week or the month they launched an app, and thereby track daily, weekly or monthly cohorts.


* **Behavioral Cohorts**: divide users by the behaviors they have (or haven’t) taken in your app within a given time period. These could be any number of discrete actions that a user can perform – App Install, App Launch, App Uninstall, Transaction or Charged, or any combination of these actions / events.

In this project, we will divide customers into acquisition cohorts based on initial purchase from the online Retail II dataset. The dataset contains all sales transactions for UK-based and registered customers. The company mainly sells unique all-occasion gift-ware. We will investigate customer retention rate by monthly cohort with each cohort of customers grouped by initial purchase.

The dataset contains 1,067,371 rows and 8 columns.

* InvoiceNo: Invoice number
* StockCode: Product (item) code
* Description: Product (item) name
* Quantity: The quantities of each product (item) per transaction
* InvoiceDate: Invoice date and time
* UnitPrice: Unit price. Product price per unit in sterling (Â£).
* CustomerID: Customer number
* Country: Country name

In [ ]:
# import libraries
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', 100)
from pandas_profiling import ProfileReport 
%matplotlib inline

# import data
df_retail = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")

In [ ]:
df_retail.head()

In [ ]:
df_retail.info()

# Exploration Data

In [ ]:
# Variables & Distribution
df_retail.profile_report()

According to the overview above, there are 247389 rows missing values and 25061 duplicate rows. 'Description' and 'CustomerID' have the most missing values. So, I'll remove some rows from 'CustomerID' that have a high percentage of missing values. 

We will remove 25061 duplicate rows from the dataset. Several columns data type will be changed. And then, We will add a necessary column for further analysis, such as the 'Total Sales' column, which is calculated from the 'Quantity' and 'Price' calculations. 

In case we have a dataset with a time span of 2010-2011. We assume that the first purchase occurred in January 2010. The dataset for the 2009 year is incomplete because it only includes data in December. So, we will exclude it.

In [ ]:
# drop missing value in 'Customer ID' column
df_retail.dropna(subset=['Customer ID'],inplace=True)

In [ ]:
# change column data type
df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate'])
df_retail.sort_values(by=['InvoiceDate'])
df_retail['Customer ID'] = df_retail['Customer ID'].astype(np.int64)

# add 'Total' column
df_retail['Total Sales'] = df_retail['Quantity'] * df_retail['Price']

#exclude data in 2009
df_retail = df_retail[df_retail['InvoiceDate'].dt.year != 2009]

In [ ]:
# check duplicate rows
df_retail[df_retail.duplicated()]

In [ ]:
# drop duplicate rows
df_retail.drop_duplicates(inplace=True)
df_retail.shape

In [ ]:
#check negative quantity
df_retail[df_retail['Quantity'] < 0]


The dataset have negative quantities which means some of orders are most return. We will remove it because we won't be necessary it for this analysis.

In [ ]:
# drop negative quantity
df_retail.drop(df_retail[df_retail['Quantity'] < 0].index, inplace=True)

In [ ]:
df_retail.info()

## Number of Transaction

In [ ]:
# extract hour, day, and month
df_retail['dayofweek'] = df_retail['InvoiceDate'].dt.strftime("%A")
df_retail['hour'] = df_retail['InvoiceDate'].dt.strftime('%H')
df_retail['month'] = df_retail['InvoiceDate'].dt.strftime('%Y-%m')

In [ ]:
df_retail['dayofweek'].unique()

In [ ]:
# pivot number of transaction per hour of day and day of the week
categories = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday']
tr_count_pivot = df_retail.pivot_table(index='dayofweek', columns='hour', values='Invoice', aggfunc='nunique').fillna(0).loc[categories, (slice(None))]
print(tr_count_pivot)

In [ ]:
# number of transaction heatmap
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(tr_count_pivot,
            annot=True, 
            fmt='g', 
            cmap='YlGnBu',
            vmin=0.0,
            vmax=1200,
            ax=ax)
ax.set_title('Number of Transaction By Per Hour of Day In Day of The Week', fontsize=16)
ax.set(xlabel='Hour of Day', ylabel='Day of The Week')
plt.tight_layout()
plt.show()

It's interesting because there is no transaction on Saturday. As we can see, the most transactions occurred between 10 am and 15 pm. The peak transactions occurred on Wednesday at 12 pm.

In [ ]:
# count transaction by distinct customer ID
count_tr = df_retail.groupby(['month', 'Customer ID'])['Invoice'].nunique().reset_index()
print(count_tr)

In [ ]:
month_tr = count_tr.groupby(['month'])['Invoice'].sum().reset_index()
print(month_tr)

In [ ]:
# monthly number of transaction lineplot
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(data=month_tr,
             x=month_tr['month'],
             y=month_tr['Invoice'],
             marker='o', 
             linestyle='-')
ax.set_title('Monthly Number of Transaction', fontsize=16)
ax.set(xlabel='Month', ylabel='Transaction')
# label points on the lineplot
for x, y in zip(month_tr['month'], month_tr['Invoice']):
    plt.text(x = x,
             y = y+45,
             s = '{:.0f}'.format(y),
             color = 'red')
    # add set_backgroundcolor(‘color’) after plt.text(‘…’)
    plt.text(x, y+45, '{:.0f}'.format(y), color='white').set_backgroundcolor('#965786')
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

We notice that the curve has a similar pattern. It consistently peaks in March, June, September, and October. The highest monthly number of transactions occurred in November, followed by a fall in December. Perhaps the increase in transactions is related to the Holiday Season.

## Monthly Active Users

In [ ]:
# assign monthly active user
mau = df_retail.groupby(['month'])['Customer ID'].nunique().reset_index()
print(mau)

In [ ]:
# monthly customer active lineplot
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(data=mau,
             x=mau['month'],
             y=mau['Customer ID'],
             marker='o', 
             linestyle='-')
ax.set_title('Monthly Customer Active', fontsize=16)
ax.set(xlabel='Month', ylabel='Customer')
# label points on the lineplot
for x, y in zip(mau['month'], mau['Customer ID']):
    plt.text(x = x,
             y = y,
             s = '{:.0f}'.format(y),
             color = 'red')
    # add set_backgroundcolor(‘color’) after plt.text(‘…’)
    plt.text(x, y, '{:.0f}'.format(y), color='white').set_backgroundcolor('#965786')
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

The above curve pattern is the same as the number of transaction curve. It is caused by customers being more active in making purchases throughout the Holiday Season. The highest monthly active users were in November 2010 and November 2011.

## Cohort Analysis

We will build cohort analysis based on what I've learned on [here](https://campus.datacamp.com/courses/customer-segmentation-in-python/). First, we will assign acquisition month cohort each customer in order to create a cohort month. Next, we will calculate the time offset by extracting integer values of the date for cohort index. Then, we will calculate the number of monthly active customers in each cohort. The cohort table is being plotted in a heatmap.



In [ ]:
# assign acquisition month cohort
def get_month(x): 
    return dt.datetime(x.year, x.month, 1)

df_retail = df_retail[df_retail['InvoiceDate'].dt.year != 2009]
df_retail['InvoiceMonth'] = df_retail['InvoiceDate'].apply(get_month)
group = df_retail.groupby('Customer ID')['InvoiceMonth']
df_retail['CohortMonth'] = group.transform('min').dt.to_period('M')
df_retail.head()

In [ ]:
# assign time offset value
def get_date_int(df_retail, column):
    year = df_retail[column].dt.year
    month = df_retail[column].dt.month
    day = df_retail[column].dt.day
    return year, month, day

# Getting the integers for date parts from the `InvoiceMonth` column
transcation_year, transaction_month, _ = get_date_int(df_retail, 'InvoiceMonth')
# Getting the integers for date parts from the `CohortMonth` column
cohort_year, cohort_month, _ = get_date_int(df_retail, 'CohortMonth')
#  Get the  difference in years
years_diff = transcation_year - cohort_year
# Calculate difference in months
months_diff = transaction_month - cohort_month
""" Extract the difference in months from all previous values
 "+1" in addeded at the end so that first month is marked as 1 instead of 0 for easier interpretation. 
 """
df_retail['CohortIndex'] = years_diff * 12 + months_diff  + 1 
df_retail.head()

In [ ]:
# Counting monthly active user from each chort
grouping = df_retail.groupby(['CohortMonth', 'CohortIndex'])
# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['Customer ID'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()
# Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'Customer ID')
# Print top 5 rows of Dataframe
cohort_data.head()

In [ ]:
# Coverting the retention rate into percentage and rounding off.
cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)
retention_rate = retention.round(3)*100

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(15, 10), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_rate, 
                mask=retention_rate.isnull(), 
                annot=True, 
                fmt='g', 
                cmap='RdYlGn',
                vmin=0.0, 
                vmax=50,
                ax=ax[1])
    ax[1].set_title('Customer Retention Rate by Monthly Cohorts', fontsize=16)
    ax[1].set(xlabel='Cohort Index',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_sizes).rename(columns={1: 'Cohort Size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()
plt.show()

# Conclusion

Based on our findings, we will summarize insights that we've learned:

* The most transactions occurred between 10 am and 15 pm. The peak transactions occurred on Wednesday at 12 pm.
* The highest monthly number of transactions occurred in November due to the Holiday Season.
* The highest monthly active users were in November 2010 and November 2011.
* As we can see, the customers from January and February 2010 have a better retention rate overall, and the longer the customers have been with the online retail.
* 47.2% of cohorts that purchased in January 2010 were still active 3 months later. This suggests that the online retail is successful in retaining 47.2% of its customers.
* Customers who remained active for 3 months were retained. To build customer retention strategies, we can create a remarketing campaign such as free delivery, coupons, loyalty rewards, and so on. So, after 3 months, we can keep customers coming back for repeat purchases.